In [5]:
import json
import requests as req
import numpy as np
import pandas as pd
from os import makedirs
from datetime import date, timedelta
from os.path import join, exists

In [6]:
#test url
url = ("http://content.guardianapis.com/search?from-date=2016-03-04&to-date=2016-03-04&order-by=newest&show-fields=all&page-size=200&api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27")
response = req.get(url)
query = url
print(query)

http://content.guardianapis.com/search?from-date=2016-03-04&to-date=2016-03-04&order-by=newest&show-fields=all&page-size=200&api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27


In [59]:
#Test variables
base_url = "http://content.guardianapis.com/search?q="
query_term = "Procter & Gamble Company (The)"
sort = "&order-by=oldest&show-fields=all&page-size=50&"
from_date = "from-date=2017-01-01&"
to_date = "to-date=2017-12-31&"
api_key = "api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27"

query_url = base_url + query_term + sort + from_date + to_date + api_key

response = req.get(query_url)

data = response.json()
print(query_url)

http://content.guardianapis.com/search?q=Procter & Gamble Company (The)&order-by=oldest&show-fields=all&page-size=50&from-date=2017-01-01&api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27


In [61]:
# TEST article count
article = data['response']['results'] 
article_count = len(article)
print(article_count)

50


In [62]:
cd vaderSentiment_master

C:\Users\craig\Desktop\PythonCode\python-challenge\corporate_failure_bounce\vaderSentiment_master


In [63]:
cd vaderSentiment

C:\Users\craig\Desktop\PythonCode\python-challenge\corporate_failure_bounce\vaderSentiment_master\vaderSentiment


In [64]:
import vaderSentiment

In [65]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [66]:
# Test Vader Analysis without  loop

Article_Sentiment = (article[1]['fields']['body'])
compound = analyzer.polarity_scores(Article_Sentiment)["compound"]
pos = analyzer.polarity_scores(Article_Sentiment)["pos"]
neu = analyzer.polarity_scores(Article_Sentiment)["neu"]
neg = analyzer.polarity_scores(Article_Sentiment)["neg"]
            
            # Print Analysis
print("Compound Score: %s" % compound)
print("Positive Score: %s" % pos)
print("Neutral Score: %s" % neu)
print("Negative Score: %s" % neg)

Compound Score: -0.9966
Positive Score: 0.072
Neutral Score: 0.831
Negative Score: 0.097


In [51]:
Sentiment_Summation = compound 
print(Sentiment_Summation)

0.999


In [71]:
cd ..

C:\Users\craig\Desktop\PythonCode\python-challenge\corporate_failure_bounce


In [72]:
# Import csv to stock_price_pd
stock_price_pd = pd.read_csv("price drop 3.csv")
stock_price_pd.head()

,Ticker,Name,Opening Price,Closing Price,Week 0,Week 1
0,AAP,Advance Auto Parts Inc,35.51,26.97,10/10/08,10/17/08
1,AAP,Advance Auto Parts Inc,85.57,66.68,5/18/12,5/25/12
2,AAP,Advance Auto Parts Inc,198.71,161.62,11/13/15,11/20/15
3,ABC,AmerisourceBergen Corporation (Holding Co),36.14,29.45,10/10/08,10/17/08
4,ABC,AmerisourceBergen Corporation (Holding Co),36.31,18.31,6/19/09,6/26/09


In [73]:
#Add columns for Article Description
stock_price_pd["Article_Count"] = ""
stock_price_pd["Sentiment_Summation"] = ""
stock_price_pd["Article_Avg_Sentiment"] = ""
stock_price_pd.head()

,Ticker,Name,Opening Price,Closing Price,Week 0,Week 1,Article_Count,Sentiment_Summation,Article_Avg_Sentiment
0,AAP,Advance Auto Parts Inc,35.51,26.97,10/10/08,10/17/08,,,
1,AAP,Advance Auto Parts Inc,85.57,66.68,5/18/12,5/25/12,,,
2,AAP,Advance Auto Parts Inc,198.71,161.62,11/13/15,11/20/15,,,
3,ABC,AmerisourceBergen Corporation (Holding Co),36.14,29.45,10/10/08,10/17/08,,,
4,ABC,AmerisourceBergen Corporation (Holding Co),36.31,18.31,6/19/09,6/26/09,,,


In [74]:
# Loop through tickers and articles

# Counter
row_count = 0

#Test variables
base_url = "http://content.guardianapis.com/search?q="
sort = "&order-by=oldest&show-fields=all&page-size=50&"
api_key = "api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27"

query_url = base_url + query_term + sort + from_date + to_date + api_key

response = req.get(query_url)

data = response.json()
print(query_url)


for index, row in stock_price_pd.iterrows():
    query_term = stock_price_pd["Name"]
    Open_Date = stock_price_pd["Week 0"]
    Close_Date = stock_price_pd["Week 1"]
    from_date = "from-date="+str(Open_Date)+"&"
    to_date = "to-date="+str(Close_Date)+"&"

    
    query_url = base_url + query_term + sort + from_date + to_date + api_key
    print(query)

    #Get article data
    data = response.json()
    article = data['response']['results'] 
    article_count = len(article)
    
    # Store the article count into the Data Frame
    stock_price_pd.set_value(index, "Article_Count", article_count)
                                             
    # TEST article count



    # Run Vader Analysis with While loop
    Sentiment_Summation = 0 
    for x in range(0,article_count):
        Article_Sentiment = (article[x]['fields']['body'])
        compound = analyzer.polarity_scores(Article_Sentiment)["compound"]
        pos = analyzer.polarity_scores(Article_Sentiment)["pos"]
        neu = analyzer.polarity_scores(Article_Sentiment)["neu"]
        neg = analyzer.polarity_scores(Article_Sentiment)["neg"]

        Sentiment_Summation += compound 
    
    try: 
        stock_price_pd.set_value(index, "Sentiment_Summation", Sentiment_Summation)
      
    
    except:
        #print("Error with data. Skipping")
        continue

    # Reset row count
    row_count = 0

http://content.guardianapis.com/search?q=Procter & Gamble Company (The)&order-by=oldest&show-fields=all&page-size=50&from-date=2017-01-01&to-date=2017-12-31&api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27
http://content.guardianapis.com/search?from-date=2016-03-04&to-date=2016-03-04&order-by=newest&show-fields=all&page-size=200&api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27
http://content.guardianapis.com/search?from-date=2016-03-04&to-date=2016-03-04&order-by=newest&show-fields=all&page-size=200&api-key=8a2fcf01-9a95-4f45-a42f-f467c8bdab27


KeyboardInterrupt: 